# Analyzing marketing data for personal loans

## Background

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

## Problem

I have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data. I need to design the schema in a way that would allow data from future campaigns to be easily imported.

They have supplied the current campaign data as a csv file called `"bank_marketing.csv"`

## Solution

I clean, reformat, and split their source data in order to populate the database. I then provide SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, I provide the database design script as a `.sql` file that they can then run themselves.

### Database schema

## client

| column           | data type | description                                                |
| ---------------- | --------- | ---------------------------------------------------------- |
| `id`             | `serial`  | Client ID - primary key                                    |
| `age`            | `integer` | Client's age in years                                      |
| `job`            | `text`    | Client's type of job                                       |
| `marital`        | `text`    | Client's marital status                                    |
| `education`      | `text`    | Client's level of education                                |
| `credit_default` | `boolean` | Whether the client's credit is in default                  |
| `housing`        | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan`           | `boolean` | Whether the client has an existing personal loan           |

<br>

## campaign

| column                       | data type | description                                                                          |
| ---------------------------- | --------- | ------------------------------------------------------------------------------------ |
| `campaign_id`                | `serial`  | Campaign ID - primary key                                                            |
| `client_id`                  | `serial`  | Client ID - references `id` in the `client` table                                    |
| `number_contacts`            | `integer` | Number of contact attempts to the client in the current campaign                     |
| `contact_duration`           | `integer` | Last contact duration in seconds                                                     |
| `pdays`                      | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign                    |
| `previous_outcome`           | `boolean` | Outcome of the previous campaign                                                     |
| `campaign_outcome`           | `boolean` | Outcome of the current campaign                                                      |
| `last_contact_date`          | `date`    | Last date the client was contacted                                                   |

<br>

## economics

| column                 | data type | description                                                              |
| ---------------------- | --------- | ------------------------------------------------------------------------ |
| `client_id`            | `serial`  | Client ID - references `id` in the `client` table                        |
| `emp_var_rate`         | `float`   | Employment variation rate (quarterly indicator)                          |
| `cons_price_idx`       | `float`   | Consumer price index (monthly indicator)                                 |
| `euribor_three_months` | `float`   | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed`      | `float`   | Number of employees (quarterly indicator)                                |


In [ ]:
import pandas as pd
import numpy as np

# Start coding here...
df_bank = pd.read_csv('bank_marketing.csv')
# df_bank.head()
client = df_bank[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
client.rename(columns={'client_id': 'id'}, inplace=True)

campaign = df_bank[['client_id', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']]
campaign.rename(columns={'duration': 'contact_duration', 'previous': 'previous_campaign_contacts', 'y': 'campaign_outcome', 'poutcome': 'previous_outcome', 'campaign': 'number_contacts'}, inplace=True)

economics = df_bank[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]
economics.rename(columns={'euribor3m': 'euribor_three_months', 'nr_employed': 'number_employed'}, inplace=True)

client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.nan)
client['job'] = client['job'].str.replace('.', '')

campaign['previous_outcome'] = campaign['previous_outcome'].replace({'success': 1, 'failure': 0, 'nonexistent': np.nan})
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace({'success': 1, 'failure': 0})

campaign['campaign_id'] = 1
campaign['last_contact_date'] = pd.to_datetime('2022-' + df_bank['month'].astype(str) + '-' + df_bank['day'].astype(str))

client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

client_table = """CREATE TABLE client (
id SERIAL PRIMARY KEY not null,
age INT,
job TEXT,
marital TEXT,
education TEXT,
credit_default BOOLEAN,
housing BOOLEAN,
loan BOOLEAN
)
\COPY client FROM 'client.csv' DELIMITER ',' CSV HEADER
;"""

campaign_table = """CREATE TABLE campaign (
campaign_id SERIAL PRIMARY KEY not null,
client_id SERIAL references client (id),
number_contacts INT,
contact_duration INT,
pdays INT,
previous_campaign_contacts INT,
previous_outcome BOOLEAN,
campaign_outcome BOOLEAN,
last_contact_date DATE
)
\COPY campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER
;"""


economics_table = """CREATE TABLE economics (
client_id SERIAL references client (id),
emp_var_rate FLOAT,
cons_price_idx FLOAT,
euribor_three_months FLOAT,
number_employed FLOAT
)
\COPY economics FROM 'economics.csv' DELIMITER ',' CSV HEADER
;"""